# 使用chroma测试小说《围城》数据集

## 准备

In [1]:
%%time

# 导入需要的包

from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.llms.openai_like import OpenAILike

import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

CPU times: user 3.01 s, sys: 336 ms, total: 3.34 s
Wall time: 3.03 s


In [2]:
Settings.chunk_size=128
Settings.chunk_overlap=10

Settings

_Settings(_llm=None, _embed_model=None, _callback_manager=None, _tokenizer=None, _node_parser=SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7fe4b53671f0>, id_func=<function default_id_func at 0x7fe575ab2dd0>, chunk_size=128, chunk_overlap=10, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?'), _prompt_helper=None, _transformations=None)

In [3]:
%%time

# 初始化全局 llm

llm = OpenAILike(model="xiaoyu", 
                 api_base="http://192.168.0.72:3000/v1", 
                 api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
                 is_chat_model=True,
                 temperature=0.1
                )

Settings.llm =llm

CPU times: user 121 ms, sys: 0 ns, total: 121 ms
Wall time: 121 ms


In [4]:
%%time

# 初始化全局 embedding 模型

ollama_embedding = OllamaEmbedding(
    model_name="dztech/bge-large-zh:v1.5",
    # model_name="bge-m3:latest",
    base_url="http://192.168.0.72:11435",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.embed_model = ollama_embedding

CPU times: user 551 ms, sys: 31.5 ms, total: 582 ms
Wall time: 581 ms


In [5]:
%%time

import logging
import sys

# 设置系统日志，便于设置level排查

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

CPU times: user 217 µs, sys: 24 µs, total: 241 µs
Wall time: 243 µs


## 索引

In [6]:
%%time

# load documents
documents = SimpleDirectoryReader("./books/").load_data()
print("Document ID:", documents[0].doc_id)

Document ID: e0591074-14b1-4f39-903f-e046aaf4da32
CPU times: user 5.17 ms, sys: 5.76 ms, total: 10.9 ms
Wall time: 10.3 ms


In [8]:
%%time

# create client and a new collection
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("quickstart")

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
CPU times: user 82.8 ms, sys: 7.85 ms, total: 90.7 ms
Wall time: 91.2 ms


In [9]:
%%time

# set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

CPU times: user 20.4 s, sys: 596 ms, total: 21 s
Wall time: 1min 27s


## 查询

### 基本嵌入查询

In [10]:
%%time

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    similarity_cutoff=0.5
)

CPU times: user 35.9 ms, sys: 0 ns, total: 35.9 ms
Wall time: 35.4 ms


In [11]:
%%time

streaming_response = query_engine.query("方鸿渐的妻子是谁")
streaming_response.print_response_stream()
print()

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
方鸿渐的妻子是柔嘉。
CPU times: user 304 ms, sys: 2.43 ms, total: 307 ms
Wall time: 19.3 s


In [12]:
%%time

streaming_response = query_engine.query("方鸿渐的父亲是谁，说出他的名字")
streaming_response.print_response_stream()
print()

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
方鸿渐的父亲是一乡之望，但具体的名字在给出的文本中并未提及。
CPU times: user 139 ms, sys: 14.8 ms, total: 153 ms
Wall time: 12.2 s


In [13]:
%%time

streaming_response = query_engine.query("方鸿渐的父亲是谁，说出他的名字，名字好像带一个翁字")
streaming_response.print_response_stream()
print()

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
方豚翁
CPU times: user 109 ms, sys: 693 µs, total: 110 ms
Wall time: 10.9 s


In [14]:
%%time

streaming_response = query_engine.query("文中提到的局部真理是啥意思")
streaming_response.print_response_stream()
print()

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.16

### rerank

In [15]:
%%time

from llama_index.core.postprocessor import SentenceTransformerRerank

reranker = SentenceTransformerRerank(model='/models/bge-reranker-v2-m3', top_n=5)

CPU times: user 1.59 s, sys: 744 ms, total: 2.33 s
Wall time: 1.53 s


In [16]:
%%time

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    node_postprocessors=[reranker],
    similarity_cutoff=0.5
)

CPU times: user 183 µs, sys: 0 ns, total: 183 µs
Wall time: 186 µs


In [18]:
%%time

streaming_response = query_engine.query("方鸿渐的妻子是谁")
streaming_response.print_response_stream()
print()

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
方鸿渐的妻子在文本中并未直接提及，但从上下文推测，她可能是周家的女性成员，因为有提到“换了旁的女人，要把鸿渐看守得牢牢的”，并且提到了周太太对他的期望。然而，具体是哪一位女性并没有直接点明，因此无法确定是哪一位“方鸿渐的好太太”。
CPU times: user 835 ms, sys: 25.3 ms, total: 860 ms
Wall time: 3.61 s


In [19]:
%%time

streaming_response = query_engine.query("方鸿渐的父亲是谁，说出他的名字")
streaming_response.print_response_stream()
print()

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
方鸿渐的父亲叫方鹏图。
CPU times: user 805 ms, sys: 56.5 ms, total: 861 ms
Wall time: 1.82 s


In [20]:
%%time

streaming_response = query_engine.query("方鸿渐的父亲是谁，说出他的名字，名字好像带一个翁字")
streaming_response.print_response_stream()
print()

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
方鸿渐的父亲叫方豚翁。
CPU times: user 849 ms, sys: 0 ns, total: 849 ms
Wall time: 1.81 s


In [21]:
%%time

streaming_response = query_engine.query("文中提到的局部真理是啥意思")
streaming_response.print_response_stream()
print()

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
文中提到的“局部真理”可能是指孙小姐说有事到教授宿舍来时，那一瞬间鸿渐对她的话并没有太在意，直到后来她有所行动或表达了某个观点，才意识到其含义或重要性。这可以理解为话语的意义在最初可能并不明显，但随着时间的推移或情境的变化，其真实含义逐渐显现出来，就像陌生的猫“喵”叫一样，让人注意到它的存在。这里的“局部真理”可能是指话语中隐藏的、需要一定时间和情境来理解和体会的部分含义。
CPU times: user 949 ms, sys: 32.6 ms, total: 981 ms
Wall time: 5.65 s
